In [1]:
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import numpy as np
import math
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib notebook
from Iterative_solvers_l0_l1 import *
import scipy.io as sio
import itertools
from operator import itemgetter, attrgetter
import matplotlib.pyplot as plt
from sklearn.linear_model import lars_path, enet_path, lasso_path
from itertools import cycle

import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression

import seaborn as sns
c = sns.color_palette("Blues",20)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
X_APR    = loadtxt('Burgers/X_burgers.dat', dtype='double')
T_APR    = loadtxt('Burgers/T_burgers.dat', dtype='double') 
X_APR.shape, T_APR.shape


((256,), (101,))

In [3]:
data = sio.loadmat('Burgers/burgers.mat')
data 

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Sat Aug 27 07:36:14 2016',
 '__version__': '1.0',
 '__globals__': [],
 't': array([[ 0. ],
        [ 0.1],
        [ 0.2],
        [ 0.3],
        [ 0.4],
        [ 0.5],
        [ 0.6],
        [ 0.7],
        [ 0.8],
        [ 0.9],
        [ 1. ],
        [ 1.1],
        [ 1.2],
        [ 1.3],
        [ 1.4],
        [ 1.5],
        [ 1.6],
        [ 1.7],
        [ 1.8],
        [ 1.9],
        [ 2. ],
        [ 2.1],
        [ 2.2],
        [ 2.3],
        [ 2.4],
        [ 2.5],
        [ 2.6],
        [ 2.7],
        [ 2.8],
        [ 2.9],
        [ 3. ],
        [ 3.1],
        [ 3.2],
        [ 3.3],
        [ 3.4],
        [ 3.5],
        [ 3.6],
        [ 3.7],
        [ 3.8],
        [ 3.9],
        [ 4. ],
        [ 4.1],
        [ 4.2],
        [ 4.3],
        [ 4.4],
        [ 4.5],
        [ 4.6],
        [ 4.7],
        [ 4.8],
        [ 4.9],
        [ 5. ],
        [ 5.1],
        [ 5.2],
        [

In [4]:
u = real(data['usol'])
u

array([[ 2.22044605e-16, -4.31008024e-11,  4.77340778e-10, ...,
         1.81398207e-04,  1.90083736e-04,  1.99013820e-04],
       [ 4.85722573e-16,  4.28848658e-11, -4.78508736e-10, ...,
         2.08914676e-04,  2.18644653e-04,  2.28635026e-04],
       [ 1.04083409e-15, -4.26225071e-11,  4.79589604e-10, ...,
         2.40188515e-04,  2.51064261e-04,  2.62217886e-04],
       ...,
       [-7.80625564e-17,  4.34146816e-11, -4.73120376e-10, ...,
         1.17517897e-04,  1.23619041e-04,  1.29915917e-04],
       [-1.82145965e-17, -4.33821173e-11,  4.74640424e-10, ...,
         1.36050551e-04,  1.42928147e-04,  1.50017788e-04],
       [ 2.08166817e-17,  4.32728853e-11, -4.76044357e-10, ...,
         1.57232607e-04,  1.64970064e-04,  1.72934908e-04]])

In [5]:
dx = X_APR[1] - X_APR[0]
dt = T_APR[1] - T_APR[0]
print(" shape of u ", u.shape)
print(" dt is ", dt, " dx is ", dx)

 shape of u  (256, 101)
 dt is  0.1  dx is  0.0625


In [6]:
numpy.random.seed(0)
noise_level = 0.01
un = u + noise_level*std(u)*np.random.randn(u.shape[0],u.shape[1])
un

array([[ 3.20002589e-03,  7.25893058e-04,  1.77545056e-03, ...,
         4.11619257e-04,  9.19300403e-04,  3.61508648e-03],
       [-2.44486163e-03, -2.30468491e-03,  1.75850435e-03, ...,
         2.63340285e-03, -4.51058513e-04, -2.05603543e-04],
       [ 1.99480429e-03,  1.18866134e-03,  1.16121175e-03, ...,
        -2.12987707e-03,  3.25894615e-03,  4.78659680e-05],
       ...,
       [ 3.27045419e-03,  2.44065572e-03, -1.76253326e-03, ...,
        -1.11024514e-03,  2.63633113e-03,  1.64602294e-03],
       [-1.24845421e-03, -1.86948200e-03,  1.29204268e-04, ...,
         1.43000350e-03, -1.63888629e-03,  2.01048503e-03],
       [ 1.62548236e-03,  4.45618436e-04,  4.14901989e-03, ...,
        -1.34135673e-03,  1.45315589e-03,  4.91202337e-03]])

In [7]:
y, X, description = build_linear_system(un, dt, dx, D=4, P=3, time_diff = 'poly', space_diff = 'poly',deg_x = 5, deg_t = 5, 
                                      width_x = 10, width_t = 10)

 The value of D is changed to 4  4


In [8]:
print(" length of description ", len(description))

 length of description  20


In [13]:
description_trunc, X_no_intercept = remove_intercept(X, description)


i : 1 u
i : 2 u^2
i : 3 u^3
i : 4 u_{x}
i : 5 uu_{x}
i : 6 u^2u_{x}
i : 7 u^3u_{x}
i : 8 u_{xx}
i : 9 uu_{xx}
i : 10 u^2u_{xx}
i : 11 u^3u_{xx}
i : 12 u_{xxx}
i : 13 uu_{xxx}
i : 14 u^2u_{xxx}
i : 15 u^3u_{xxx}
i : 16 u_{xxxx}
i : 17 uu_{xxxx}
i : 18 u^2u_{xxxx}
i : 19 u^3u_{xxxx}
 shape of the truncated description  19
shape 19116 20
shape of no_intercept (19116, 19)


/Users/anuragtrivedi/PDE-STRIDE/Iterative_solvers_l0_l1.py:35: ComplexWarning: Casting complex values to real discards the imaginary part
  X_no_intercept[:,i-1] = X[:,i]


In [16]:
X_no_intercept.shape

(19116, 19)

In [14]:
description_trunc

{0: 'u',
 1: 'u^2',
 2: 'u^3',
 3: 'u_{x}',
 4: 'uu_{x}',
 5: 'u^2u_{x}',
 6: 'u^3u_{x}',
 7: 'u_{xx}',
 8: 'uu_{xx}',
 9: 'u^2u_{xx}',
 10: 'u^3u_{xx}',
 11: 'u_{xxx}',
 12: 'uu_{xxx}',
 13: 'u^2u_{xxx}',
 14: 'u^3u_{xxx}',
 15: 'u_{xxxx}',
 16: 'uu_{xxxx}',
 17: 'u^2u_{xxxx}',
 18: 'u^3u_{xxxx}'}

In [17]:
u_t = y
u_t

array([[ 0.00428459+0.j],
       [-0.0015763 +0.j],
       [ 0.00159016+0.j],
       ...,
       [ 0.00222126+0.j],
       [ 0.00147895+0.j],
       [-0.00333055+0.j]], dtype=complex64)

In [18]:
np.random.seed(50)
reduced_size = 200

In [19]:
ordered = np.arange(0, len(u_t))
ordered 

array([    0,     1,     2, ..., 19113, 19114, 19115])

In [20]:
np.random.shuffle(ordered)

In [21]:
X_reduced = np.zeros((reduced_size, len(description_trunc)))
X_reduced.shape

(200, 19)

In [22]:
y_reduced = np.zeros((reduced_size,1))
y_reduced.shape   

(200, 1)

In [23]:
X_stand   = np.zeros((reduced_size, len(description_trunc)))
X_stand

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
y_stand   = np.zeros((reduced_size,1))
y_stand 

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [25]:
for i in range(0, reduced_size):
    X_reduced[i,:] = X_no_intercept[int(ordered[i]),:]
    y_reduced[i,:] = u_t[int(ordered[i])]


/var/folders/xl/xdhqxz854nqdvlfwhg056fv00000gn/T/ipykernel_1001/1464867284.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  y_reduced[i,:] = u_t[int(ordered[i])]


In [26]:
X_stand = preprocessing.scale(X_reduced)
y_stand = preprocessing.scale(y_reduced, with_std=False)
y_stand = y_stand.reshape(reduced_size,1)
X_stand,y_stand

(array([[ 1.81053605,  1.39316377,  0.88210424, ...,  0.43361688,
          0.26086389,  0.13280538],
        [-0.61364394, -0.42591231, -0.32992634, ..., -0.03971437,
         -0.07428563, -0.08319192],
        [-0.59998465, -0.42591149, -0.32992629, ..., -0.04057442,
         -0.07428726, -0.08319193],
        ...,
        [-0.48923629, -0.42161621, -0.32978652, ..., -0.04881687,
         -0.07459836, -0.08320174],
        [-0.60625496, -0.42592629, -0.32992631, ..., -0.039602  ,
         -0.07428555, -0.08319192],
        [-0.1442133 , -0.35928668, -0.32142801, ..., -0.09482105,
         -0.08174973, -0.08411265]]),
 array([[-5.17348739e-02],
        [-6.27820982e-04],
        [-3.20276853e-03],
        [-2.62664015e-02],
        [ 7.76371491e-02],
        [-5.27522842e-02],
        [ 3.36576216e-03],
        [ 8.41714073e-04],
        [-2.39868293e-02],
        [-4.28478624e-02],
        [-1.71072987e-03],
        [ 9.84391114e-03],
        [-2.75438474e-01],
        [-3.50332658e-

In [27]:
mul_vector = np.matmul(X_stand.transpose(), y_stand)
mul_vector  

array([[ -2.86256353],
       [ -3.77183073],
       [ -3.92161119],
       [-12.6668089 ],
       [-12.04296736],
       [-10.26597686],
       [ -8.78441109],
       [  4.19610941],
       [  1.28176115],
       [  0.55513734],
       [  0.55815544],
       [  9.40237639],
       [  9.22479358],
       [  7.55324393],
       [  6.24531522],
       [ -2.30864097],
       [  2.04085186],
       [  3.71523644],
       [  3.84673059]])

In [28]:
lambda_max = (1.0/reduced_size)*np.linalg.norm(mul_vector, np.inf)
lambda_max 

0.0633340445079832

In [29]:
M             = 20 # M in the paper

In [30]:
factor        = np.exp(np.log(10.)/M)
factor 

1.1220184543019633

In [31]:
lambda_regl   = np.zeros((M,1))
lambda_regl

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [32]:
coeff_alpha   = np.zeros((len(description_trunc),M)) # stores the output
coeff_alpha.shape

(19, 20)

In [33]:
alphas_regl = np.zeros((M,1))
alphas_regl.shape

(20, 1)

In [34]:
for alpha in range(0, M):
    alphas_regl[alpha] = lambda_max/factor**alpha

In [35]:
alphas_regl[alpha]

array([0.0071062])

In [ ]:
# normalized_alphas_sample = np.zeros((M,))
# normalized_alphas_sample[1]

In [36]:
normalized_alphas_sample = alphas_regl/alphas_regl[0]
normalized_alphas_sample 

array([[1.        ],
       [0.89125094],
       [0.79432823],
       [0.70794578],
       [0.63095734],
       [0.56234133],
       [0.50118723],
       [0.44668359],
       [0.39810717],
       [0.35481339],
       [0.31622777],
       [0.28183829],
       [0.25118864],
       [0.22387211],
       [0.19952623],
       [0.17782794],
       [0.15848932],
       [0.14125375],
       [0.12589254],
       [0.11220185]])

In [37]:
neg_log_stable_lambda = -np.log10(normalized_alphas_sample)
neg_log_stable_lambda

array([[-0.  ],
       [ 0.05],
       [ 0.1 ],
       [ 0.15],
       [ 0.2 ],
       [ 0.25],
       [ 0.3 ],
       [ 0.35],
       [ 0.4 ],
       [ 0.45],
       [ 0.5 ],
       [ 0.55],
       [ 0.6 ],
       [ 0.65],
       [ 0.7 ],
       [ 0.75],
       [ 0.8 ],
       [ 0.85],
       [ 0.9 ],
       [ 0.95]])

In [38]:
print(" lambda max is ", alphas_regl[0], " lambda min is ", alphas_regl[M-1])
print(" lambda max is ", normalized_alphas_sample[0], " lambda min is ", normalized_alphas_sample[M-1])

 lambda max is  [0.06333404]  lambda min is  [0.0071062]
 lambda max is  [1.]  lambda min is  [0.11220185]


In [39]:
p = X_stand.shape[1]
p

19

In [40]:
coefs_gihtd    = np.zeros((p, M))
coefs_gihtd.shape 

(19, 20)

### Key Point for Algorithm 

1- The algorithm iterates over the groups instead of the features.
2- The debiasing step is applied to each group individually.

In [42]:
def group_IHT(X, y, groups, lasso_lam, max_iter, sub_iter, tol, htp_flag, print_flag=False):
    """
    Group Iterative Hard Thresholding for sparse group Lasso regression.

    Parameters:
        X (np.ndarray): The design matrix.
        y (np.ndarray): The target vector.
        groups (list): A list of indices for each group.
        lasso_lam (float): The regularization parameter.
        max_iter (int): The maximum number of iterations.
        sub_iter (int): The number of sub-iterations for the de-biasing step.
        tol (float): The tolerance for convergence.

    Returns:
        coeff (np.ndarray): The sparse group Lasso coefficients.
    """

    n, d = X.shape

    coeff_old = np.zeros((1, d))
    coeff_new = np.zeros((1, d))

    # Initialize the coefficients
    init = (1. / np.linalg.norm(X.T.dot(X), 2)) * X.T.dot(y).reshape(d,)
    for g in groups:
        init[0, g] = np.mean(y)

    coeff_old[0, :] = init
    support = np.array(list(set(groups)))

    # Main loop
    for iteration in range(0, max_iter):

        # Gradient descent step
        temp = np.zeros((1, d))
        for g in groups:
            temp[0, g] = coeff_old[0, g] + (2.0 / np.linalg.norm(X[:, g].T.dot(X[:, g]), 2)) * (y - X[:, g].dot(coeff_old[0, g]))

        # Hard-thresholding of the gradient descent estimates
        smallinds = []
        biginds = []
        for g in groups:
            smallinds_g = np.where(abs(temp[0, g]) <= lasso_lam)[0]
            biginds_g = [i for i in range(len(g)) if i not in smallinds_g]
            smallinds.extend(smallinds_g)
            biginds.extend(biginds_g)

        # De-biasing step
        coeff_new = np.zeros((1, d))
        for g in groups:
            coeff_new[0, g], frac_iter = debiasing(X[:, g], y, temp[0, g], coeff_new[0, g], smallinds_g, biginds_g, sub_iter, lasso_lam)

        # Check for convergence
        if (np.linalg.norm(coeff_new[0, :] - coeff_old[0, :], np.inf) < tol) and np.all(support == biginds):
            return coeff_new[0, :]

        # Update support and coefficients
        support = biginds.copy()
        coeff_old[0, :] = coeff_new[0, :].copy()

    return coeff_new[0, :]
